# Impact Prediction in the FX Market

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

sns.set(rc = {"figure.figsize":(16, 9)})

In [8]:
class OnlineData:

    def __init__(self, filename):
        self.filename = filename
        self.f = open(filename, 'r')
        self.TotalQ = 1
        self.Qnow = np.array([0])
        self.Pnow = np.array([0])
 
    def __next__(self):
        line = next(self.f)[:-1]
        if len(line) < 1:
            return None
        Q_, P_ = np.array([z[1:-1].split(",") for z in line.split("\t")
                          ]).astype(float).astype(int).T
        self.Qnow = Q_
        self.Pnow = P_
        CS = np.cumsum(P_ * np.ediff1d(Q_, to_begin=Q_[0]))

        self.TotalQ = sum(Q_)

        def F(Q):
            if Q <= Q_[0]:
                return P_[0] * Q
            if Q > Q_[-1]:
                return None
            i = np.searchsorted(Q_, Q)
            return CS[i - 1] + (Q - Q_[i - 1]) * P_[i]

        self.F = F
        return self.F

    def __iter__(self):
        return self

    def close(self):
        self.f.close()

In [3]:
def SrtategyMI(numbers: np.array, 
               times: np.array,
               orderbook_bid: OnlineData,
               dQfilename,
               marketIfilename):
        
        i = 0

        dQ = open(dQfilename, 'w+')
        marketI = open(marketIfilename, 'w+')

        for t in range(max(times) + 1):
            BuyPrice = next(orderbook_bid)

            if t in times:
                 deltaQ = numbers[i] / orderbook_bid.TotalQ * 10000
                 MarkI = ((BuyPrice(numbers[i] + 1) - BuyPrice(numbers[i])) / BuyPrice(1) - 1) * 10000
                 dQ.write(str(deltaQ) + " ")
                 marketI.write(str(MarkI) + " ")
                 print(f"time: {t} Ask: {BuyPrice(1)} BuyPrice: {BuyPrice(numbers[i]) / numbers[i]} MI: {MarkI}")
                 i += 1


        dQ.close()
        marketI.close()
                  

In [73]:
def OrderSizes(datafilename: str,
               dQfilename: str,
               len: int):
    
    it1 = OnlineData(datafilename)
    it2 = OnlineData(datafilename)

    next(it2)

    dQ = open(dQfilename, 'w+')

    for i in range(len):
        if(it1.Pnow[0] == it2.Pnow[0] and int(it1.Qnow[0]) == int(it2.Qnow[0])):
            next(it1)
            next(it2)
            continue
        else:
            if it1.TotalQ - it2.TotalQ > 5000000:
                dQ.write(str(i) + "\n")
            
            next(it1)
            next(it2)
        
    dQ.close()
            
        

In [3]:
def MIdQ(askfilename: str,
               dQfilename: str,
               MIfilename: str,
               depth: int):
    
    it1 = OnlineData(askfilename)
    it2 = OnlineData(askfilename)

    Price2 = next(it2)
    Price1 = next(it1)
    Price2 = next(it2)

    dQ = open(dQfilename, 'w+')
    MarkI = open(MIfilename, 'w+')
    Indexes = open("indexes.txt", 'w+')


    for i in range(depth):
        if(i > 100):
            volume = it1.TotalQ - it2.TotalQ
            deltaP = Price2(1) - Price1(1)
            if volume > 0 and not Price1(volume) == None and not deltaP == None and deltaP > 0:
                tildS = Price1(volume) / volume
                if(abs(tildS - Price1(1))/Price1(1) > 0):
                    MarkI.write(str(abs(tildS - Price1(1))/Price1(1) * 10000) + ", ")
                    dQ.write(str(volume / 142266100 * 10000) + ", ")
                    Indexes.write(str(i) + ", ")
                
        
        Price1 = next(it1)
        Price2 = next(it2)
        
    dQ.close()
            

In [33]:
dQs = np.genfromtxt("deltaQseries.txt", delimiter=", ", dtype = float)
print(len(dQs))

576


In [3]:
def MIdQ(askfilename: str,
               csvfilename: str,
               depth: int):
    
    af1 = open(askfilename, 'r')
    af2 = open(askfilename, 'r')

    line1 = next(af1)[:-1]
    line2 = next(af2)[:-1]
    line2 = next(af2)[:-1]
    Q1, P1 = np.array([z[1:-1].split(",") for z in line1.split("\t")
                          ]).astype(float).astype(int).T
    
    Q2, P2 = np.array([z[1:-1].split(",") for z in line2.split("\t")
                          ]).astype(float).astype(int).T
    CS = np.cumsum(P1 * np.ediff1d(Q1, to_begin=Q1[0]))
    CS = np.cumsum(P2 * np.ediff1d(Q2, to_begin=Q2[0]))

    def F1(Q):
        if Q <= Q1[0]:
            return P1[0] * Q
        if Q > Q1[-1]:
            return None
        i = np.searchsorted(Q1, Q)
        return CS[i - 1] + (Q - Q1[i - 1]) * P1[i]

    cs = open(csvfilename, 'w+')
    cs.write("Time, Impact, DeltaQ\n")

    obnum = 1

    for i in range(depth):
        if(i > 10):
            volume = sum(Q1) - sum(Q2)
            deltaP = F2(1) - F1(1)
            if volume > 0 and not F1(volume) == None and not deltaP == None and deltaP > 0:
                tildS = F1(volume) / volume
                if(abs(tildS - F1(1))/F1(1) > 0 and abs(tildS - F1(1))/F1(1) * 10000 > 1.0):
                    cs.write(str(float(i)) + ", " + str(abs(tildS - F1(1))/F1(1) * 10000) + ", " + str(volume / 142266100 * 10000) + ", \n")
                    obnum += 1
        
        line1 = next(af1)[:-1]
        line2 = next(af2)[:-1]

        Q1, P1 = np.array([z[1:-1].split(",") for z in line1.split("\t")
                          ]).astype(float).astype(int).T
    
        Q2, P2 = np.array([z[1:-1].split(",") for z in line2.split("\t")
                            ]).astype(float).astype(int).T
        
        CS1 = np.cumsum(P1 * np.ediff1d(Q1, to_begin=Q1[0]))
        CS2 = np.cumsum(P2 * np.ediff1d(Q2, to_begin=Q2[0]))

        def F1(Q):
            if Q <= Q1[0]:
                return P1[0] * Q
            if Q > Q1[-1]:
                return None
            i = np.searchsorted(Q1, Q)
            return CS1[i - 1] + (Q - Q1[i - 1]) * P1[i]
        
        def F2(Q):
            if Q <= Q2[0]:
                return P2[0] * Q
            if Q > Q2[-1]:
                return None
            i = np.searchsorted(Q2, Q)
            return CS2[i - 1] + (Q - Q2[i - 1]) * P2[i]
    af1.close()
    af2.close()
    cs.close()


In [24]:
def equalFromPlace(list1, list2):
    flag = 0
    for i in range(len(list1) - len(list2) + 1):
        flag = 1
        for j in range(1, len(list2)):
            if(list2[j] != list1[i + j]):
                flag = 0
        
        if(flag): return flag

    return 0


In [13]:
def MIdQ(askfilename: str,
               dQfilename: str,
               depth: int):
    
    it1 = OnlineData(askfilename)
    it2 = OnlineData(askfilename)

    Price2 = next(it2)
    Price1 = next(it1)
    Price2 = next(it2)

    dQ = open(dQfilename, 'w+')


    for i in range(depth):
        if(i > 100):
            volume = it1.TotalQ - it2.TotalQ
            deltaP = Price2(1) - Price1(1)
            if volume > 0 and not Price1(volume) == None and not deltaP == None and deltaP > 0:
                tildS = Price1(volume) / volume
                if(abs(tildS - Price1(1))/Price1(1) > 0.0001):
                    dQ.write(str(i) + ", " 
                             + str(abs(tildS - Price1(1))/Price1(1) * 10000) + ", " 
                             + str(volume / 142266100 * 10000) + "\n")
                
        
        Price1 = next(it1)
        Price2 = next(it2)
        
    dQ.close()

In [14]:
MIdQ("DataForImpact/ask_22_10_03.tsv", "dQI.csv", 1000)


In [20]:
af1 = open("ask.txt", 'r')
af2 = open("ask.txt", 'r')

In [25]:
list1 = [(10,5), (9, 4), (7, 30)]
list2 = [(8, 4), (7, 15)]

print(equalFromPlace(list1, list2))

0


In [22]:
next(af1)
next(af2)

'(100000.0, 22888)\t(150000.0, 22550)\n'